### **Detailed Implementation Plan: MLflow Evaluation (3 Students)**

This notebook documents the implementation plan, phases, and responsibilities for a team working on an MLflow evaluation project. Each section corresponds to one person and their responsibilities.

### Jabir Nahari - Core Infrastructure & Basic Tracking

#### **Phase 1: Foundation Setup**

1. **Environment Configuration**

- Set up MLflow tracking server locally

1. **Dataset & Baseline Model**

- Prepare datasets
- Create baseline Random Forest and Logistic Regression models

3. **Basic MLflow Integration**

- Implement parameter logging for different algorithms
- Set up metric tracking (accuracy, precision, recall, F1)
- Create basic experiment run templates

In [31]:

import warnings
warnings.filterwarnings('ignore')

import os
import random
# Imports with checks
try:
    import pandas as pd
except ImportError as e:
    raise ImportError("pandas is required for this notebook. Install with: pip install pandas") from e


from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import mlflow
try:
    import seaborn as sns
except ImportError as e:
    raise ImportError("Seaborn is required to load the Titanic dataset.") from e

# Loading and preprocessing datasets
titanic = sns.load_dataset('titanic')
titanic = titanic.dropna(subset=['survived'])
titanic = titanic.select_dtypes(include=['number']).fillna(0)
if 'survived' not in titanic.columns:
    raise RuntimeError('Titanic dataset does not contain the required "survived" column; ensure the dataset is available and correct')
X_titanic = titanic.drop(columns=['survived'])
y_titanic = titanic['survived']

iris = load_iris(as_frame=True)
X_iris = iris.data
y_iris = iris.target

# training and loggin all in one function
def train_and_log_baselines(X, y, dataset_name='dataset', seed=42):
    import mlflow
    from mlflow.models.signature import infer_signature
    import numpy as np
    
    TRACKING_URI = 'http://127.0.0.1:5000'
    mlflow.set_tracking_uri(TRACKING_URI)
    print('MLflow tracking URI ->', mlflow.get_tracking_uri())

    experiment_name = f"{dataset_name} experiment"
    try:
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        print(f'Warning: could not set experiment "{experiment_name}": {e}')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

    models = {
        'random_forest': RandomForestClassifier(random_state=seed),
        'logistic_regression': LogisticRegression(max_iter=500)
    }

    for name, model in models.items():
        with mlflow.start_run(nested=True):
            mlflow.log_param('model', name)
            mlflow.log_param('dataset', dataset_name)
            mlflow.log_param('seed', seed)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
            mlflow.log_metrics({'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1})
            try:
                try:
                    input_example = X_train.head(5)
                except Exception:
                    input_example = pd.DataFrame(np.array(X_train)[:5])
                try:
                    sample_X = input_example
                    sample_preds = model.predict(sample_X)
                    signature = infer_signature(sample_X, sample_preds)
                except Exception:
                    signature = None
                import mlflow.sklearn
                try:
                    mlflow.sklearn.log_model(model, name='./artifacts/', signature=signature, input_example=input_example)
                except TypeError:
                    try:
                        mlflow.sklearn.log_model(model, artifact_path='./artifacts/', signature=signature, input_example=input_example)
                    except Exception:
                        raise
            except Exception:
                try:
                    import mlflow.sklearn
                    mlflow.sklearn.log_model(model, artifact_path='./artifacts/')
                except Exception:
                    pass

# Generate a list of random seeds for robustness testing
seeds = [random.randint(0, 10000) for _ in range(5)]
print(f'Using seeds for robustness testing: {seeds}')

# Train and log for Iris and Titanic datasets across seeds
for seed in seeds:
    train_and_log_baselines(X_iris, y_iris, dataset_name='iris', seed=seed)
    train_and_log_baselines(X_titanic, y_titanic, dataset_name='titanic', seed=seed)


2025/11/10 08:40:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Using seeds for robustness testing: [474, 9067, 1332, 4836, 783]
MLflow tracking URI -> http://127.0.0.1:5000


2025/11/10 08:40:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run smiling-bird-686 at: http://127.0.0.1:5000/#/experiments/1/runs/4526281b91734b3fbc5c9f0ca37c7a27
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run monumental-koi-42 at: http://127.0.0.1:5000/#/experiments/1/runs/64dd574b2de14c729541be201951a26b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
MLflow tracking URI -> http://127.0.0.1:5000


2025/11/10 08:40:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/10 08:40:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

🏃 View run defiant-fawn-524 at: http://127.0.0.1:5000/#/experiments/2/runs/d7102b1cc35945a58fac14126f577232
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run resilient-kit-741 at: http://127.0.0.1:5000/#/experiments/2/runs/6cd145be75fc48ddb53ab7090289bfcd
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
MLflow tracking URI -> http://127.0.0.1:5000


2025/11/10 08:40:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/10 08:40:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run enthused-ant-453 at: http://127.0.0.1:5000/#/experiments/1/runs/84cd53c3f0db46058a990fdf12ea48a5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run popular-crane-46 at: http://127.0.0.1:5000/#/experiments/1/runs/dda0b64b81ba4ffcb93067ac56a91fb8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
MLflow tracking URI -> http://127.0.0.1:5000


2025/11/10 08:40:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/10 08:40:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run intrigued-horse-46 at: http://127.0.0.1:5000/#/experiments/2/runs/0e05dd4c984c4e49844aba18a9ad19de
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run sassy-seal-92 at: http://127.0.0.1:5000/#/experiments/2/runs/75ab9f5d708b4e66b6d2ea93edf4a84e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


#### **Phase 2: Performance Benchmarking**

1. **Scalability Testing**

- Test with different dataset sizes
- Measure training time vs. data volume
- Log resource usage (CPU, memory)

2. **Hyperparameter Optimization Tracking**

- Implement automated hyperparameter sweeps
- Compare multiple algorithm performances
- Generate performance comparison charts

In [32]:

# Performance Benchmarking (scalability using synthetic data)
from sklearn.datasets import make_classification
import time
sizes = [1000, 10000, 100000]
results = []

experiment_name = 'Scalability Benchmarking Experiment'
try:
    mlflow.set_experiment(experiment_name)
except Exception as e:
    print(f'Warning: could not set experiment "{experiment_name}": {e}')

for seed in seeds:
    for n in sizes:
        X_s, y_s = make_classification(n_samples=n, n_features=20, random_state=seed)
        Xtr, Xte, ytr, yte = train_test_split(X_s, y_s, test_size=0.2, random_state=seed)
        clf = RandomForestClassifier(n_estimators=10, random_state=seed)
        start = time.time()
        clf.fit(Xtr, ytr)
        elapsed = time.time() - start
        ypred = clf.predict(Xte)
        acc = accuracy_score(yte, ypred)
        results.append({'seed': seed, 'n_samples': n, 'train_time_s': elapsed, 'accuracy': acc})
        try:
            with mlflow.start_run(nested=True):
                mlflow.log_param('seed', seed)
                mlflow.log_param('n_samples', n)
                mlflow.log_metric('train_time_s', elapsed)
                mlflow.log_metric('accuracy', acc)
        except Exception:
            pass

🏃 View run bald-snake-698 at: http://127.0.0.1:5000/#/experiments/3/runs/b35413ed8d4749b18ec10d18f066ae38
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run skittish-goose-490 at: http://127.0.0.1:5000/#/experiments/3/runs/4881efa8e8744f4ba548bd5e91f6c39b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run learned-hen-46 at: http://127.0.0.1:5000/#/experiments/3/runs/d90542948c364a62967c71592a1c80da
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run nosy-mare-741 at: http://127.0.0.1:5000/#/experiments/3/runs/28256897536643f3a22e50db028b5bb0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run colorful-ram-243 at: http://127.0.0.1:5000/#/experiments/3/runs/804739924c1f4be2ad3832034e507e43
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/3
🏃 View run indecisive-pig-451 at: http://127.0.0.1:5000/#/experiments/3/runs/57d52c21c1134396a7fc45027f1a40c3
🧪 View experiment at: http://127.0.0.1:5000/#/experimen

In [33]:
# Hyperparameter optimization tracking: simple GridSearchCV example logged to MLflow
param_grid = {'n_estimators': [10, 50], 'max_depth': [None, 10]}

# Define lists for looping over different configurations
sample_sizes = [1000, 10000]
n_features_list = [10, 20]
cv_folds = [3, 5]

experiment_name = 'Grid Search Experiment'
try:
    mlflow.set_experiment(experiment_name)
except Exception as e:
    print(f'Warning: could not set experiment "{experiment_name}": {e}')

for seed in seeds:
    for sample_size in sample_sizes:
        for n_features in n_features_list:
            for cv_fold in cv_folds:
                grid = GridSearchCV(RandomForestClassifier(random_state=seed), param_grid, cv=cv_fold, n_jobs=-1)
                X_small, y_small = make_classification(n_samples=sample_size, n_features=n_features, random_state=seed)
                grid.fit(X_small, y_small)
                try:
                    with mlflow.start_run(run_name=f'grid_search_s{sample_size}_f{n_features}_cv{cv_fold}_seed{seed}'):
                        mlflow.log_param('seed', seed)
                        mlflow.log_param('sample_size', sample_size)
                        mlflow.log_param('n_features', n_features)
                        mlflow.log_param('cv_folds', cv_fold)
                        mlflow.log_params(grid.best_params_)
                        mlflow.log_metric('best_score', grid.best_score_)
                        mlflow.log_metric('mean_cv_score', grid.cv_results_['mean_test_score'].mean())
                        mlflow.log_metric('std_cv_score', grid.cv_results_['std_test_score'].mean())
                        mlflow.log_metric('best_index', grid.best_index_)
                        cv_results = pd.DataFrame(grid.cv_results_)
                        csv_filename = f'temp/grid_search_s{sample_size}_f{n_features}_cv{cv_fold}_seed{seed}.csv'
                        cv_results.to_csv(csv_filename,index=False)
                        try:
                            mlflow.log_artifact(csv_filename)
                        except Exception:
                            pass
                except Exception:
                    pass

print('Scalability results (first 3):', results[:3])
print('Grid search completed for all configurations.')

🏃 View run grid_search_s1000_f10_cv3_seed474 at: http://127.0.0.1:5000/#/experiments/4/runs/a207e78041f64f9aa3eb92253434898a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
🏃 View run grid_search_s1000_f10_cv5_seed474 at: http://127.0.0.1:5000/#/experiments/4/runs/9037cb5965d9419a9c49c532a20f6d61
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
🏃 View run grid_search_s1000_f20_cv3_seed474 at: http://127.0.0.1:5000/#/experiments/4/runs/acc286a3d39f4ff0b187abf174059186
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
🏃 View run grid_search_s1000_f20_cv5_seed474 at: http://127.0.0.1:5000/#/experiments/4/runs/84cebd3f7861452b99c350d187528ff5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
🏃 View run grid_search_s10000_f10_cv3_seed474 at: http://127.0.0.1:5000/#/experiments/4/runs/5ab3bccd38ea4bd6b28396836dee84d9
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/4
🏃 View run grid_search_s10000_f10_cv5_seed474 at: http://127.0.0.1:5000/#/

In [34]:
import os
import shutil

# Remove all files and subdirectories inside the 'temp' folder
if os.path.exists('temp'):
    for filename in os.listdir('temp'):
        file_path = os.path.join('temp', filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                print(f'Deleted file: {file_path}')
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
                print(f'Deleted directory: {file_path}')
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
else:
    print('temp folder does not exist')

Deleted file: temp/grid_search_s10000_f20_cv5_seed9067.csv
Deleted file: temp/grid_search_s1000_f10_cv5_seed783.csv
Deleted file: temp/grid_search_s10000_f20_cv3_seed783.csv
Deleted file: temp/grid_search_s1000_f10_cv5_seed9067.csv
Deleted file: temp/grid_search_s10000_f10_cv5_seed474.csv
Deleted file: temp/grid_search_s1000_f20_cv3_seed474.csv
Deleted file: temp/grid_search_s10000_f20_cv3_seed9067.csv
Deleted file: temp/grid_search_s1000_f10_cv3_seed9067.csv
Deleted file: temp/grid_search_s10000_f20_cv5_seed783.csv
Deleted file: temp/grid_search_s10000_f10_cv3_seed9067.csv
Deleted file: temp/grid_search_s1000_f10_cv3_seed783.csv
Deleted file: temp/grid_search_s1000_f20_cv5_seed474.csv
Deleted file: temp/grid_search_s1000_f20_cv3_seed9067.csv
Deleted file: temp/grid_search_s10000_f10_cv3_seed474.csv
Deleted file: temp/grid_search_s10000_f10_cv5_seed9067.csv
Deleted file: temp/grid_search_s1000_f20_cv5_seed9067.csv
Deleted file: temp/grid_search_s1000_f10_cv5_seed474.csv
Deleted file: t

In [35]:
# MLflow client: point to the MLflow server and run the baseline logging remotely
import mlflow
from mlflow.exceptions import MlflowException

MLFLOW_SERVER = 'http://127.0.0.1:5000'  
mlflow.set_tracking_uri(MLFLOW_SERVER)
print('MLflow tracking URI ->', mlflow.get_tracking_uri())
try:
    for seed in seeds:
        train_and_log_baselines(X_iris, y_iris, dataset_name='iris', seed=seed)
        train_and_log_baselines(X_titanic, y_titanic, dataset_name='titanic', seed=seed)
    print('Baseline runs logged to remote MLflow server.')
except NameError as e:
    print('Required data or helper function not found in scope:', e)
except Exception as e:
    print('Error while running remote logging:', e)

2025/11/10 08:42:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


MLflow tracking URI -> http://127.0.0.1:5000
MLflow tracking URI -> http://127.0.0.1:5000
🏃 View run abundant-wasp-504 at: http://127.0.0.1:5000/#/experiments/1/runs/45c52e362b0d4bf89b55430d6d5ed7a8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


2025/11/10 08:42:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run merciful-yak-334 at: http://127.0.0.1:5000/#/experiments/1/runs/318743e215f941818a3d71c82505d242
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
MLflow tracking URI -> http://127.0.0.1:5000


2025/11/10 08:42:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/10 08:42:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

🏃 View run defiant-shark-579 at: http://127.0.0.1:5000/#/experiments/2/runs/f8ef1fdf7c624d55bbc273d3b91aa00a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run abundant-slug-557 at: http://127.0.0.1:5000/#/experiments/2/runs/a871a3e43a084bc4ace3a4cf7491889c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
MLflow tracking URI -> http://127.0.0.1:5000


2025/11/10 08:42:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/10 08:42:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run bemused-stoat-383 at: http://127.0.0.1:5000/#/experiments/1/runs/e2f68244e12b436a91dbdc8ba6183819
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run learned-moose-861 at: http://127.0.0.1:5000/#/experiments/1/runs/7d125a2b195a4e43be8b0b6a80ca9b95
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
MLflow tracking URI -> http://127.0.0.1:5000


2025/11/10 08:42:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/10 08:42:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run magnificent-bass-779 at: http://127.0.0.1:5000/#/experiments/2/runs/47f0a615deec4687bb3ff2623480aeae
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
🏃 View run fun-slug-171 at: http://127.0.0.1:5000/#/experiments/2/runs/9aa1aeb806d4456cb3a98a1a46b6980d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2
Baseline runs logged to remote MLflow server.


---

### **Hasan Al Hasan - Advanced Features & Model Management**

#### **Phase 1: Research & Advanced Setup**

1. **MLflow Components Deep Dive**

- Study Model Registry, Projects, and Models components
- Research MLflow's deployment capabilities
- Compare tracking UI alternatives

2. **Advanced Environment Setup**

- Set up MLflow Model Registry
- Configure artifact storage (local + cloud options)
- Prepare Docker environment for model serving

In [36]:
# Phase 1 - Research & Advanced Setup (Hasan)
# - Read MLflow Model Registry and Projects documentation
# - Design artifact storage strategy (local vs cloud)
# - Draft Dockerfile and serving container notes


#### **Phase 2: Advanced Feature Implementation**

1. **Model Registry & Versioning**

- Implement model versioning and staging
- Set up transition workflows (Staging → Production)

2. **Model Serving & Deployment**

- Create REST API endpoints for model serving
- Test batch and real-time inference
- Demonstrate model A/B testing capabilities

3. **Artifact Management**

- Log confusion matrices, ROC curves, feature importance
- Store custom visualizations and reports
- Implement artifact versioning

In [37]:
# Phase 2 - Advanced Feature Implementation (Hasan)
# - Implement model registration and versioning steps
# - Prepare serving scripts and simple endpoint tests
# - Add utilities to log artifacts (plots, metrics) to MLflow



---

### Nawaf Al Harthi - Research, Analysis & Integration

#### **Phase 1: Competitive Analysis**

1. **Framework Comparison Research**

- Deep comparison: MLflow vs. Kubeflow vs. Weights & Biases
- Feature comparison matrix
- Use case suitability analysis

2. **Integration Planning**

- Design demo workflow that integrates all components
- Plan presentation narrative and flow
- Create evaluation criteria for critical analysis

#### **Phase 2: Integration & Quality Assurance**

1. **End-to-End Workflow Integration**

- Combine tracking with registry and serving components
- Create comprehensive demo scenario
- Ensure seamless handoffs between components

2. **Testing & Validation**

- Validate all MLflow components work together
- Test edge cases and error handling
- Performance validation across different scenarios

3. **Documentation & Best Practices**

- Create installation and troubleshooting guide
- Document team workflows and collaboration patterns
- Prepare code documentation and comments

#### **Phase 3: Critical Analysis & Presentation**

1. **Comprehensive Evaluation**

- Analyze MLflow strengths and weaknesses
- Compare with alternative tools
- Provide improvement recommendations

---

### **Additional Tasks:**
- Comprehensive tool comparison
- End-to-end workflow integration testing
- Quality assurance across all components
- Documentation and best practices guide
- Presentation narrative and demo script coordination

In [38]:
# Phase 1 - Competitive Analysis (Nawaf)
# - Create a comparison matrix for MLflow, Kubeflow, and W&B
# - Capture features, strengths, and limitations for each


In [39]:
# Phase 2 - Integration & Quality Assurance (Nawaf)
# - Draft end-to-end demo steps that wire tracking -> registry -> serving
# - Write basic validation checks and error handling tests


In [40]:
# Phase 3 - Critical Analysis & Presentation (Nawaf)
# - Summarize findings, recommendations, and demo scripts
# - Prepare presentation notes and rehearsal checklist
